# Lab Two: Classification

Matt Farrow, Amber Clark, Blake Freeman, Megan Ball

## **2015 Flight Delays and Cancellations**
Data Source: [Kaggle](https://www.kaggle.com/usdot/flight-delays?select=flights.csv)

Our data set consists of over 5 million rows of flight information in the domestic United States for the year of 2015. In order to optimize our modeling time, we have narrowed the scope of our classification tasks to the Dallas area only (Dallas Love Field and DFW airports). 

The goal for this project is to build two classification models to predict the following for the DFW area:
1. Whether or not the flight was cancelled (our binary classification model)
2. For delayed flights, how long was the delay (in terms of groups - our multi-class classification model).

Because the scope of our two classification models is different, we will create two data sets for each modelling task.

# 1. Data Preparation

- **[10 points]** Define and prepare your class variables. Use proper variable representations (int, float, one-hot, etc.). Use pre-processing methods (as needed) for dimensionality reduction, scaling, etc. Remove variables that are not needed/useful for the analysis.

In [1]:
# Load libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
# from datetime import datetime
import altair as alt
import datetime

In [2]:
# Due to the way these columns are formatted, we want to keep the leading zeros during our import. Later on will convert them to a time format.
dtype_t = {'SCHEDULED_DEPARTURE': str,
           'DEPARTURE_TIME': str,
           'WHEELS_OFF': str,
           'SCHEDULED_TIME': str,
           'WHEELS_ON': str,
           'SCHEDULED_ARRIVAL': str,
           'ARRIVAL_TIME': str
          }

# Read in the data directly
airlines = pd.read_csv('../Data/airlines.csv')
airports = pd.read_csv('../Data/airports.csv')
flights  = pd.read_csv('../Data/flights.csv', dtype = dtype_t)

# Read in the data directly from GitHub
# airlines = pd.read_csv('https://raw.githubusercontent.com/mattfarrow1/7331-machine-learning-1/main/Data/airlines.csv')
# airports = pd.read_csv('https://raw.githubusercontent.com/mattfarrow1/7331-machine-learning-1/main/Data/airports.csv')
# flights  = pd.read_csv('https://media.githubusercontent.com/media/mattfarrow1/7331-machine-learning-1/main/Data/flights.csv', dtype = dtype_t)

/Users/mattfarrow/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3166: DtypeWarning: Columns (7,8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# Rename columns in preparation for merge
airlines.rename(columns={'IATA_CODE': 'AIRLINE_CODE'}, inplace=True)
flights.rename(columns={'AIRLINE': 'AIRLINE_CODE'}, inplace=True)

# Merge data together
df = pd.merge(flights, airlines, on='AIRLINE_CODE', how = 'left')

In [4]:
# Check variable types
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5819079 entries, 0 to 5819078
Data columns (total 32 columns):
 #   Column               Dtype  
---  ------               -----  
 0   YEAR                 int64  
 1   MONTH                int64  
 2   DAY                  int64  
 3   DAY_OF_WEEK          int64  
 4   AIRLINE_CODE         object 
 5   FLIGHT_NUMBER        int64  
 6   TAIL_NUMBER          object 
 7   ORIGIN_AIRPORT       object 
 8   DESTINATION_AIRPORT  object 
 9   SCHEDULED_DEPARTURE  object 
 10  DEPARTURE_TIME       object 
 11  DEPARTURE_DELAY      float64
 12  TAXI_OUT             float64
 13  WHEELS_OFF           object 
 14  SCHEDULED_TIME       object 
 15  ELAPSED_TIME         float64
 16  AIR_TIME             float64
 17  DISTANCE             int64  
 18  WHEELS_ON            object 
 19  TAXI_IN              float64
 20  SCHEDULED_ARRIVAL    object 
 21  ARRIVAL_TIME         object 
 22  ARRIVAL_DELAY        float64
 23  DIVERTED             int64  
 24

### 1.1 Subset to DFW area

This step will reduce our data down to flights departing from Dallas area airports only.

In [5]:
df = df[(df.ORIGIN_AIRPORT == 'DFW') | (df.ORIGIN_AIRPORT == 'DAL') | 
        (df.DESTINATION_AIRPORT == 'DFW') | (df.DESTINATION_AIRPORT == 'DAL')]

In [6]:
df.shape

(598535, 32)

In [7]:
#check airline counts
df['AIRLINE'].value_counts()

American Airlines Inc.          268550
American Eagle Airlines Inc.    107226
Southwest Airlines Co.          105345
Atlantic Southeast Airlines      49291
Spirit Air Lines                 17047
Delta Air Lines Inc.             13066
Virgin America                   10099
Skywest Airlines Inc.             8265
United Air Lines Inc.             7015
US Airways Inc.                   6162
Alaska Airlines Inc.              2698
Frontier Airlines Inc.            2575
JetBlue Airways                   1196
Name: AIRLINE, dtype: int64

American Airlines is the max class (as expected), but we do still have a decent sample size for some of the smaller airlines. We have reduced the data down from 5 million+ rows to 598,535 rows which is a much more manageable size (although still a lot of data!).

In [8]:
#check cancellations
df['CANCELLED'].value_counts()

0    583787
1     14748
Name: CANCELLED, dtype: int64

This is an unbalanced data set for one of our desired classification items, which is whether or not the flight is cancelled. When we are doing our test and training splits, we can either under or oversample our data, or choose a different cut-off value to optimize sensitivity and specificity in this binary classification.

### 1.2 Create New Variables

The data contains several timestamp and continuous variables that add additional complexity. In order to simplify the data, we created buckets to analyze similar attributes together.

We first convert SCHEDULED_DEPARTURE, DEPARTURE_TIME, ARRIVAL_TIME, and SCHEDULED_ARRIVAL into buckets based on their timestamp.

    Overnight: 12:00am - 3:59am
    Morning: 4:00am - 10:59am
    Afternoon: 11:00am - 3:59pm
    Evening: 4:01pm - 11:59pm

[Flight] DISTANCE is also divided into buckets.

    Short: 1-99 miles
    Medium: 100-999 miles
    Long: 1,000+ miles



In [9]:
#Convert times into buckets for morning, afternoon, and evening as most models cannot handle timestamps.
cut_labels = ['overnight', 'morning', 'afternoon', 'evening']
cut_bins = [0, 400, 1100, 1600, 2359]

df['SCHED_DEPARTURE_TIME'] = pd.cut(df['SCHEDULED_DEPARTURE'].astype(float), 
                                    bins=cut_bins, 
                                    labels=cut_labels, 
                                    include_lowest=True)
df['ACTUAL_DEPARTURE_TIME'] = pd.cut(df['DEPARTURE_TIME'].astype(float), 
                                     bins=cut_bins, 
                                     labels=cut_labels, 
                                     include_lowest=True)
df['SCHED_ARRIVAL_TIME'] = pd.cut(df['SCHEDULED_ARRIVAL'].astype(float), 
                                  bins=cut_bins, 
                                  labels=cut_labels, 
                                  include_lowest=True)
df['ACTUAL_ARRIVAL_TIME'] = pd.cut(df['ARRIVAL_TIME'].astype(float), 
                                  bins=cut_bins, 
                                  labels=cut_labels, 
                                  include_lowest=True)

# Bucket Flight Distance
distance_labels = ['Short', 'Medium', 'Long']
distance_bins   = [1, 100, 1000, np.inf]
df['DISTANCE_BUCKET'] = pd.cut(df['DISTANCE'],
                               bins=distance_bins,
                               labels=distance_labels)

# Look at our data with the buckets
pd.set_option('display.max_columns', None)
df.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE_CODE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,AIRLINE,SCHED_DEPARTURE_TIME,ACTUAL_DEPARTURE_TIME,SCHED_ARRIVAL_TIME,ACTUAL_ARRIVAL_TIME,DISTANCE_BUCKET
8,2015,1,1,4,AA,1112,N3LAAA,SFO,DFW,0030,0019,-11.0,17.0,0036,195,193.0,173.0,1464,0529,3.0,0545,0532,-13.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,American Airlines Inc.,overnight,overnight,morning,morning,Long
22,2015,1,1,4,NK,214,N632NK,LAS,DFW,0103,0102,-1.0,13.0,0115,147,147.0,128.0,1055,0523,6.0,0530,0529,-1.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,Spirit Air Lines,overnight,overnight,morning,morning,Long
41,2015,1,1,4,NK,972,N606NK,PHX,DFW,0159,0158,-1.0,11.0,0209,123,125.0,103.0,868,0452,11.0,0502,0503,1.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,Spirit Air Lines,overnight,overnight,morning,morning,Medium
42,2015,1,1,4,AA,2459,N3BDAA,PHX,DFW,0200,NaN,NaN,NaN,NaN,120,NaN,NaN,868,NaN,NaN,0500,NaN,NaN,0,1,B,NaN,NaN,NaN,NaN,NaN,American Airlines Inc.,overnight,NaN,morning,NaN,Medium
70,2015,1,1,4,AA,1057,N3ASAA,DFW,MIA,0515,0703,108.0,15.0,0718,161,155.0,133.0,1121,1031,7.0,0856,1038,102.0,0,0,NaN,0.0,0.0,0.0,0.0,102.0,American Airlines Inc.,morning,morning,morning,morning,Long


In [10]:
# Check how many of each bin we have
sched_depart = df['SCHED_DEPARTURE_TIME'].value_counts()
actual_depart = df['ACTUAL_DEPARTURE_TIME'].value_counts()
sched_arrival = df['SCHED_ARRIVAL_TIME'].value_counts()
sched_arrival = df['ACTUAL_ARRIVAL_TIME'].value_counts()
dist_bucket = df['DISTANCE_BUCKET'].value_counts()

print(sched_depart, '\n\n', actual_depart, '\n\n', sched_arrival, '\n\n', dist_bucket)

morning      215255
evening      191908
afternoon    188790
overnight      2582
Name: SCHED_DEPARTURE_TIME, dtype: int64 

 morning      203882
evening      193829
afternoon    183284
overnight      3360
Name: ACTUAL_DEPARTURE_TIME, dtype: int64 

 evening      262011
afternoon    181603
morning      127464
overnight     12143
Name: ACTUAL_ARRIVAL_TIME, dtype: int64 

 Medium    420466
Long      174846
Short       3223
Name: DISTANCE_BUCKET, dtype: int64


In [11]:
# Create a new column where the arrival_delay > 0 means it's delayed(=1) and if <= 0 it's not delayed(=0)
get_delay = lambda x: 0 if x <= 0 else 1
df['DELAYED'] = df.ARRIVAL_DELAY.apply(get_delay)
df.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE_CODE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,AIRLINE,SCHED_DEPARTURE_TIME,ACTUAL_DEPARTURE_TIME,SCHED_ARRIVAL_TIME,ACTUAL_ARRIVAL_TIME,DISTANCE_BUCKET,DELAYED
8,2015,1,1,4,AA,1112,N3LAAA,SFO,DFW,0030,0019,-11.0,17.0,0036,195,193.0,173.0,1464,0529,3.0,0545,0532,-13.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,American Airlines Inc.,overnight,overnight,morning,morning,Long,0
22,2015,1,1,4,NK,214,N632NK,LAS,DFW,0103,0102,-1.0,13.0,0115,147,147.0,128.0,1055,0523,6.0,0530,0529,-1.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,Spirit Air Lines,overnight,overnight,morning,morning,Long,0
41,2015,1,1,4,NK,972,N606NK,PHX,DFW,0159,0158,-1.0,11.0,0209,123,125.0,103.0,868,0452,11.0,0502,0503,1.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,Spirit Air Lines,overnight,overnight,morning,morning,Medium,1
42,2015,1,1,4,AA,2459,N3BDAA,PHX,DFW,0200,NaN,NaN,NaN,NaN,120,NaN,NaN,868,NaN,NaN,0500,NaN,NaN,0,1,B,NaN,NaN,NaN,NaN,NaN,American Airlines Inc.,overnight,NaN,morning,NaN,Medium,1
70,2015,1,1,4,AA,1057,N3ASAA,DFW,MIA,0515,0703,108.0,15.0,0718,161,155.0,133.0,1121,1031,7.0,0856,1038,102.0,0,0,NaN,0.0,0.0,0.0,0.0,102.0,American Airlines Inc.,morning,morning,morning,morning,Long,1


### 1.2 Process Dates & Times

First we create a date column for joining up to Dallas weather information.

In [12]:
# Source: https://stackoverflow.com/questions/54487059/pandas-how-to-create-a-single-date-column-from-columns-containing-year-month
df['FLIGHT_DATE'] = pd.to_datetime([f'{y}-{m}-{d}' for y, m, d in zip(df.YEAR, df.MONTH, df.DAY)])

Next we'll write a function to convert the numeric time strings into datetime format.

In [13]:
# Creating a function to change the way of representation of time in the column
def fun_format_time(hours):
        if hours == 2400:
            hours = 0
        else:
            hours = "{0:04d}".format(int(hours))
            Hourmin = datetime.time(int(hours[0:2]), int(hours[2:4]))
            return Hourmin

In [14]:
# Define the time columns
cols = ["SCHEDULED_DEPARTURE", 
        "DEPARTURE_TIME", 
        "SCHEDULED_ARRIVAL", 
        "SCHEDULED_TIME",
        "ARRIVAL_TIME",
        "WHEELS_ON",
        "WHEELS_OFF"]

In [15]:
# Look at the data prior to conversion
df[cols].head()

,SCHEDULED_DEPARTURE,DEPARTURE_TIME,SCHEDULED_ARRIVAL,SCHEDULED_TIME,ARRIVAL_TIME,WHEELS_ON,WHEELS_OFF
8,0030,0019,0545,195,0532,0529,0036
22,0103,0102,0530,147,0529,0523,0115
41,0159,0158,0502,123,0503,0452,0209
42,0200,NaN,0500,120,NaN,NaN,NaN
70,0515,0703,0856,161,1038,1031,0718


We noticed that record 42 has NaN where times should be. This is an example of a cancelled flight. We'll need to have our function exclude those when it processes the columns. 

In [16]:
# Convert times to float in order to correctly process them through the function
df[cols] = df[cols].astype(float)

In [17]:
# Run times through the new function
# Code adapted from: https://stackoverflow.com/questions/35232705/how-to-test-for-nans-in-an-apply-function-in-pandas
df['SCHEDULED_DEPARTURE'] = df['SCHEDULED_DEPARTURE'].apply(lambda x: fun_format_time(x) if pd.notnull(x) else x)
df['DEPARTURE_TIME']      = df['DEPARTURE_TIME'].apply(lambda x: fun_format_time(x) if pd.notnull(x) else x)
df['SCHEDULED_ARRIVAL']   = df['SCHEDULED_ARRIVAL'].apply(lambda x: fun_format_time(x) if pd.notnull(x) else x)
df['ARRIVAL_TIME']        = df['ARRIVAL_TIME'].apply(lambda x: fun_format_time(x) if pd.notnull(x) else x)
# df['SCHEDULED_TIME']      = df['SCHEDULED_TIME'].apply(lambda x: fun_format_time(x) if pd.notnull(x) else x)
df['WHEELS_ON']           = df['WHEELS_ON'].apply(lambda x: fun_format_time(x) if pd.notnull(x) else x)
df['WHEELS_OFF']          = df['WHEELS_OFF'].apply(lambda x: fun_format_time(x) if pd.notnull(x) else x)

In [18]:
df[cols].head()

,SCHEDULED_DEPARTURE,DEPARTURE_TIME,SCHEDULED_ARRIVAL,SCHEDULED_TIME,ARRIVAL_TIME,WHEELS_ON,WHEELS_OFF
8,00:30:00,00:19:00,05:45:00,195.0,05:32:00,05:29:00,00:36:00
22,01:03:00,01:02:00,05:30:00,147.0,05:29:00,05:23:00,01:15:00
41,01:59:00,01:58:00,05:02:00,123.0,05:03:00,04:52:00,02:09:00
42,02:00:00,NaN,05:00:00,120.0,NaN,NaN,NaN
70,05:15:00,07:03:00,08:56:00,161.0,10:38:00,10:31:00,07:18:00


Now that we have the times correctly formatted, let's combine the `FLIGHT_DATE` feature with `SCHEDULED_DEPARTURE` and `SCHEDULED_ARRIVAL` to create a complete datetime feature for each.

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 598535 entries, 8 to 5819038
Data columns (total 39 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   YEAR                   598535 non-null  int64         
 1   MONTH                  598535 non-null  int64         
 2   DAY                    598535 non-null  int64         
 3   DAY_OF_WEEK            598535 non-null  int64         
 4   AIRLINE_CODE           598535 non-null  object        
 5   FLIGHT_NUMBER          598535 non-null  int64         
 6   TAIL_NUMBER            597521 non-null  object        
 7   ORIGIN_AIRPORT         598535 non-null  object        
 8   DESTINATION_AIRPORT    598535 non-null  object        
 9   SCHEDULED_DEPARTURE    598535 non-null  object        
 10  DEPARTURE_TIME         584355 non-null  object        
 11  DEPARTURE_DELAY        584391 non-null  float64       
 12  TAXI_OUT               583924 non-null  flo

In [20]:
# Source: https://stackoverflow.com/questions/17978092/combine-date-and-time-columns-using-python-pandas

# Combine date & time for departure and arrival
df['SCHEDULED_DEPARTURE_DT'] = pd.to_datetime(df['FLIGHT_DATE'].astype(str) + ' ' + df['SCHEDULED_DEPARTURE'].astype(str))
df['SCHEDULED_ARRIVAL_DT']   = pd.to_datetime(df['FLIGHT_DATE'].astype(str) + ' ' + df['SCHEDULED_ARRIVAL'].astype(str))

In [21]:
# Look at the columns to make sure everything looks correct
df.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE_CODE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,AIRLINE,SCHED_DEPARTURE_TIME,ACTUAL_DEPARTURE_TIME,SCHED_ARRIVAL_TIME,ACTUAL_ARRIVAL_TIME,DISTANCE_BUCKET,DELAYED,FLIGHT_DATE,SCHEDULED_DEPARTURE_DT,SCHEDULED_ARRIVAL_DT
8,2015,1,1,4,AA,1112,N3LAAA,SFO,DFW,00:30:00,00:19:00,-11.0,17.0,00:36:00,195.0,193.0,173.0,1464,05:29:00,3.0,05:45:00,05:32:00,-13.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,American Airlines Inc.,overnight,overnight,morning,morning,Long,0,2015-01-01,2015-01-01 00:30:00,2015-01-01 05:45:00
22,2015,1,1,4,NK,214,N632NK,LAS,DFW,01:03:00,01:02:00,-1.0,13.0,01:15:00,147.0,147.0,128.0,1055,05:23:00,6.0,05:30:00,05:29:00,-1.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,Spirit Air Lines,overnight,overnight,morning,morning,Long,0,2015-01-01,2015-01-01 01:03:00,2015-01-01 05:30:00
41,2015,1,1,4,NK,972,N606NK,PHX,DFW,01:59:00,01:58:00,-1.0,11.0,02:09:00,123.0,125.0,103.0,868,04:52:00,11.0,05:02:00,05:03:00,1.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,Spirit Air Lines,overnight,overnight,morning,morning,Medium,1,2015-01-01,2015-01-01 01:59:00,2015-01-01 05:02:00
42,2015,1,1,4,AA,2459,N3BDAA,PHX,DFW,02:00:00,NaN,NaN,NaN,NaN,120.0,NaN,NaN,868,NaN,NaN,05:00:00,NaN,NaN,0,1,B,NaN,NaN,NaN,NaN,NaN,American Airlines Inc.,overnight,NaN,morning,NaN,Medium,1,2015-01-01,2015-01-01 02:00:00,2015-01-01 05:00:00
70,2015,1,1,4,AA,1057,N3ASAA,DFW,MIA,05:15:00,07:03:00,108.0,15.0,07:18:00,161.0,155.0,133.0,1121,10:31:00,7.0,08:56:00,10:38:00,102.0,0,0,NaN,0.0,0.0,0.0,0.0,102.0,American Airlines Inc.,morning,morning,morning,morning,Long,1,2015-01-01,2015-01-01 05:15:00,2015-01-01 08:56:00


### 1.3 Append Dallas-Area Weather

We download historical weather data for the DFW area from [Open Weather](https://openweathermap.org) and will append it to our data set in order to use it with our models.

In [22]:
# Read in the data
import datetime
weather = pd.read_csv('../Data/dfw_weather.csv')
weather['dt_iso'] = weather['dt_iso'].astype(str)

In [23]:
# Remove "+0000 UTC"
weather['dt_iso_update'] = weather['dt_iso'].str.split('+').str[0]

In [24]:
# Look at the data
weather.info()
weather.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68238 entries, 0 to 68237
Data columns (total 26 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   dt                   68238 non-null  int64  
 1   dt_iso               68238 non-null  object 
 2   timezone             68238 non-null  int64  
 3   city_name            68238 non-null  object 
 4   lat                  68238 non-null  float64
 5   lon                  68238 non-null  float64
 6   temp                 68238 non-null  float64
 7   feels_like           68238 non-null  float64
 8   temp_min             68238 non-null  float64
 9   temp_max             68238 non-null  float64
 10  pressure             68238 non-null  int64  
 11  sea_level            0 non-null      float64
 12  grnd_level           0 non-null      float64
 13  humidity             68238 non-null  int64  
 14  wind_speed           68238 non-null  float64
 15  wind_deg             68238 non-null 

,dt,dt_iso,timezone,city_name,lat,lon,temp,feels_like,temp_min,temp_max,pressure,sea_level,grnd_level,humidity,wind_speed,wind_deg,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_id,weather_main,weather_description,weather_icon,dt_iso_update
0,1388534400,2014-01-01 00:00:00 +0000 UTC,-21600,Dallas,32.776664,-96.796988,46.98,35.71,42.80,50.00,1025,NaN,NaN,37,11.50,160,NaN,NaN,NaN,NaN,20,801,Clouds,few clouds,02n,2014-01-01 00:00:00
1,1388538000,2014-01-01 01:00:00 +0000 UTC,-21600,Dallas,32.776664,-96.796988,44.51,32.25,40.64,48.20,1024,NaN,NaN,46,13.80,140,NaN,NaN,NaN,NaN,1,800,Clear,sky is clear,01n,2014-01-01 01:00:00
2,1388541600,2014-01-01 02:00:00 +0000 UTC,-21600,Dallas,32.776664,-96.796988,42.76,32.70,39.00,46.40,1024,NaN,NaN,42,9.22,160,NaN,NaN,NaN,NaN,1,800,Clear,sky is clear,01n,2014-01-01 02:00:00
3,1388545200,2014-01-01 03:00:00 +0000 UTC,-21600,Dallas,32.776664,-96.796988,42.58,31.23,37.40,46.94,1024,NaN,NaN,42,11.50,170,NaN,NaN,NaN,NaN,1,800,Clear,sky is clear,01n,2014-01-01 03:00:00
4,1388548800,2014-01-01 04:00:00 +0000 UTC,-21600,Dallas,32.776664,-96.796988,41.85,29.91,37.04,44.96,1023,NaN,NaN,45,12.66,160,NaN,NaN,NaN,NaN,1,800,Clear,sky is clear,01n,2014-01-01 04:00:00


In [25]:
# Convert new column to a datetime type
weather['date_time'] =  pd.to_datetime(weather['dt_iso_update'], format='%Y-%m-%d %H:%M')

In [26]:
# Drop unnecessary column
weather_drop = ['dt', 'dt_iso', 'timezone', 'city_name', 'lat', 'lon', 'weather_icon']
weather = weather.drop(columns = weather_drop)

In [27]:
# Convert all columns to uppercase to match flight data
weather.columns = weather.columns.str.upper()

In [28]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68238 entries, 0 to 68237
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   TEMP                 68238 non-null  float64       
 1   FEELS_LIKE           68238 non-null  float64       
 2   TEMP_MIN             68238 non-null  float64       
 3   TEMP_MAX             68238 non-null  float64       
 4   PRESSURE             68238 non-null  int64         
 5   SEA_LEVEL            0 non-null      float64       
 6   GRND_LEVEL           0 non-null      float64       
 7   HUMIDITY             68238 non-null  int64         
 8   WIND_SPEED           68238 non-null  float64       
 9   WIND_DEG             68238 non-null  int64         
 10  RAIN_1H              7593 non-null   float64       
 11  RAIN_3H              1261 non-null   float64       
 12  SNOW_1H              247 non-null    float64       
 13  SNOW_3H              43 non-nul

In [29]:
# Merge df & weather data sets
df = pd.merge(df, weather.add_suffix('_O'), how='inner', left_on='SCHEDULED_DEPARTURE_DT', right_on='DATE_TIME_O')
df = pd.merge(df, weather.add_suffix('_D'), how='inner', left_on='SCHEDULED_ARRIVAL_DT', right_on='DATE_TIME_D')

In [30]:
# Drop datetime columns from merged data set
datetime_drop = ['FLIGHT_DATE', 
                 'SCHEDULED_DEPARTURE_DT', 
                 'SCHEDULED_ARRIVAL_DT', 
                 'DT_ISO_UPDATE_O',
                 'DT_ISO_UPDATE_D',
                 'DATE_TIME_O',
                 'DATE_TIME_D']

df = df.drop(columns = datetime_drop)

In [31]:
# Look at the completed data set
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10120 entries, 0 to 10119
Data columns (total 74 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   YEAR                   10120 non-null  int64   
 1   MONTH                  10120 non-null  int64   
 2   DAY                    10120 non-null  int64   
 3   DAY_OF_WEEK            10120 non-null  int64   
 4   AIRLINE_CODE           10120 non-null  object  
 5   FLIGHT_NUMBER          10120 non-null  int64   
 6   TAIL_NUMBER            10103 non-null  object  
 7   ORIGIN_AIRPORT         10120 non-null  object  
 8   DESTINATION_AIRPORT    10120 non-null  object  
 9   SCHEDULED_DEPARTURE    10120 non-null  object  
 10  DEPARTURE_TIME         9754 non-null   object  
 11  DEPARTURE_DELAY        9754 non-null   float64 
 12  TAXI_OUT               9747 non-null   float64 
 13  WHEELS_OFF             9747 non-null   object  
 14  SCHEDULED_TIME         10120 non-null 

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE_CODE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,AIRLINE,SCHED_DEPARTURE_TIME,ACTUAL_DEPARTURE_TIME,SCHED_ARRIVAL_TIME,ACTUAL_ARRIVAL_TIME,DISTANCE_BUCKET,DELAYED,TEMP_O,FEELS_LIKE_O,TEMP_MIN_O,TEMP_MAX_O,PRESSURE_O,SEA_LEVEL_O,GRND_LEVEL_O,HUMIDITY_O,WIND_SPEED_O,WIND_DEG_O,RAIN_1H_O,RAIN_3H_O,SNOW_1H_O,SNOW_3H_O,CLOUDS_ALL_O,WEATHER_ID_O,WEATHER_MAIN_O,WEATHER_DESCRIPTION_O,TEMP_D,FEELS_LIKE_D,TEMP_MIN_D,TEMP_MAX_D,PRESSURE_D,SEA_LEVEL_D,GRND_LEVEL_D,HUMIDITY_D,WIND_SPEED_D,WIND_DEG_D,RAIN_1H_D,RAIN_3H_D,SNOW_1H_D,SNOW_3H_D,CLOUDS_ALL_D,WEATHER_ID_D,WEATHER_MAIN_D,WEATHER_DESCRIPTION_D
0,2015,1,1,4,AA,2459,N3BDAA,PHX,DFW,02:00:00,NaN,NaN,NaN,NaN,120.0,NaN,NaN,868,NaN,NaN,05:00:00,NaN,NaN,0,1,B,NaN,NaN,NaN,NaN,NaN,American Airlines Inc.,overnight,NaN,morning,NaN,Medium,1,34.88,26.22,33.44,36.75,1032,NaN,NaN,60,6.93,20,NaN,NaN,NaN,NaN,90,804,Clouds,overcast clouds,35.11,27.99,33.98,37.04,1031,NaN,NaN,66,4.70,20,NaN,NaN,NaN,NaN,90,804,Clouds,overcast clouds
1,2015,1,1,4,AA,2307,N505AA,OMA,DFW,06:00:00,05:54:00,-6.0,30.0,06:24:00,120.0,131.0,91.0,583,07:55:00,10.0,08:00:00,08:05:00,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,American Airlines Inc.,morning,morning,morning,morning,Medium,1,34.61,27.30,33.08,36.37,1030,NaN,NaN,63,4.70,20,0.5,NaN,NaN,NaN,90,500,Rain,light rain,33.82,27.14,32.54,36.20,1029,NaN,NaN,78,4.47,28,0.5,NaN,NaN,NaN,90,500,Rain,light rain
2,2015,1,1,4,AA,1462,N547AA,ONT,DFW,06:00:00,NaN,NaN,NaN,NaN,180.0,NaN,NaN,1188,NaN,NaN,11:00:00,NaN,NaN,0,1,B,NaN,NaN,NaN,NaN,NaN,American Airlines Inc.,morning,NaN,morning,NaN,Long,1,34.61,27.30,33.08,36.37,1030,NaN,NaN,63,4.70,20,0.5,NaN,NaN,NaN,90,500,Rain,light rain,33.33,25.84,32.00,34.82,1027,NaN,NaN,78,5.82,10,0.8,NaN,NaN,NaN,90,500,Rain,light rain
3,2015,1,1,4,AA,1241,N852AA,MCO,DFW,09:00:00,08:57:00,-3.0,13.0,09:10:00,180.0,171.0,146.0,985,10:36:00,12.0,11:00:00,10:48:00,-12.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,American Airlines Inc.,morning,morning,morning,morning,Medium,0,33.44,25.97,32.00,35.56,1029,NaN,NaN,78,5.82,30,0.3,1.15,NaN,NaN,90,500,Rain,light rain,33.33,25.84,32.00,34.82,1027,NaN,NaN,78,5.82,10,0.8,NaN,NaN,NaN,90,500,Rain,light rain
4,2015,1,1,4,WN,16,N646SW,HOU,DAL,10:00:00,09:56:00,-4.0,7.0,10:03:00,60.0,51.0,41.0,239,10:44:00,3.0,11:00:00,10:47:00,-13.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,Southwest Airlines Co.,morning,morning,morning,morning,Medium,0,32.95,25.54,32.00,34.72,1027,NaN,NaN,81,5.82,80,0.5,NaN,0.3,NaN,90,500,Rain,light rain,33.33,25.84,32.00,34.82,1027,NaN,NaN,78,5.82,10,0.8,NaN,NaN,NaN,90,500,Rain,light rain


### 1.4 Missing Values

In [ ]:
#check missing values
df.isnull().sum()

In [ ]:
#check cancellations
df['CANCELLED'].value_counts()

In [ ]:
#remove non-critical columns WHEELS_ON and WHEELS_OFF
df = df.drop(['WHEELS_ON','WHEELS_OFF'], axis=1)

In [ ]:
#add category
df['ACTUAL_DEPARTURE_TIME'] = df['ACTUAL_DEPARTURE_TIME'].cat.add_categories(['N'])
df['ACTUAL_ARRIVAL_TIME'] = df['ACTUAL_ARRIVAL_TIME'].cat.add_categories(['N'])

#fill missing values with 'N' for 'N/A'
df['ACTUAL_DEPARTURE_TIME'] = df['ACTUAL_DEPARTURE_TIME'].fillna('N')
df['ACTUAL_ARRIVAL_TIME'] = df['ACTUAL_ARRIVAL_TIME'].fillna('N')

In [ ]:
# Convert missing values to 'N' for 'N/A'
df['CANCELLATION_REASON'] = df['CANCELLATION_REASON'].fillna('N')

# Update missing values in times to 0. 
# Will be updating times to a binary (1 = yes action happened, 0 = no action happened)
df['DEPARTURE_TIME'] = df['DEPARTURE_TIME'].fillna(0)

# Change all non-null values to 1
df.loc[(df.DEPARTURE_TIME != '0'), 'DEPARTURE_TIME'] = 1

# Change column name to 'DEPARTED'
df.rename(columns={'DEPARTURE_TIME': 'DEPARTED'}, inplace=True)

# Update remaining columns using same logic
cols = ['ARRIVAL_TIME']
df[cols] = df[cols].fillna(0)
df.loc[(df.ARRIVAL_TIME != '0'), 'ARRIVAL_TIME'] = 1
df.rename(columns={'ARRIVAL_TIME': 'ARRIVED'}, inplace=True)

# Fill missing values with 0
cols = ['AIR_SYSTEM_DELAY','SECURITY_DELAY','AIRLINE_DELAY','LATE_AIRCRAFT_DELAY','WEATHER_DELAY']
df[cols] = df[cols].fillna(0)

# Change remaining null values to 0 if flight was cancelled
df.loc[(df.CANCELLED == 1), ('DEPARTURE_DELAY', 'TAXI_OUT', 'ELAPSED_TIME','AIR_TIME','TAXI_IN','ARRIVAL_DELAY')] = 0

In [ ]:
#check missing values left
df.isnull().sum()

In [ ]:
#drop remaining missing values and check total cancels left
df = df.dropna()
df['CANCELLED'].value_counts()

### 1.5 Log Transformations

As we discovered during Lab 1, the DEPARTURE_DELAY, ARRIVAL_DELAY, DISTANCE, TAXI_IN, ELAPSED_TIME, and AIR_TIME variables are siginificantly right-skewed with a large number of outliers. In order to normalize these values, we did a log transformation.

In [ ]:
min(df["DEPARTURE_DELAY"])

In [ ]:
min(df["ARRIVAL_DELAY"])

In [ ]:
min(df["DISTANCE"])

In [ ]:
min(df["TAXI_IN"])

In [ ]:
min(df["ELAPSED_TIME"])

In [ ]:
min(df["AIR_TIME"])

MB comment: Because we have negative values in the departure and arrival delays, the below code that we have been using overwrites any negative value to 0. I have modified the code to keep 0 if the value is 0, but take the log for everything else.

In [ ]:
# Log transformation keeping the 0 in the data sets 
#df["DEPARTURE_DELAY_log"] = df["DEPARTURE_DELAY"].map(lambda i: np.log1p(i) if i > 0 else 0) 
#df["ARRIVAL_DELAY_Log"]   = df["ARRIVAL_DELAY"].map(lambda i: np.log1p(i) if i > 0 else 0)
#df["DISTANCE_log"]        = df["DISTANCE"].map(lambda i: np.log1p(i) if i > 0 else 0) 
#df["TAXI_IN_Log"]         = df["TAXI_IN"].map(lambda i: np.log1p(i) if i > 0 else 0)
#df["ELAPSED_TIME_log"]    = df["ELAPSED_TIME"].map(lambda i: np.log1p(i) if i > 0 else 0) 
#df["AIR_TIME_log"]        = df["AIR_TIME"].map(lambda i: np.log1p(i) if i > 0 else 0)

In [ ]:
# Log transformation keeping the 0 in the data sets. Because we have negative values, need to offset to make minimum
# equal to zero and not a negative number. For the other vars, no need to run lambda function as min > 0 which improves
# run time
df["DEPARTURE_DELAY_log"] = df["DEPARTURE_DELAY"].map(lambda i: np.log(i + 68) if i != -68 else 0) 
df["ARRIVAL_DELAY_log"]   = df["ARRIVAL_DELAY"].map(lambda i: np.log(i + 87) if i != -87 else 0)
df["DISTANCE_log"]        = np.log(df["DISTANCE"])
df["TAXI_IN_log"]         = np.log1p(df["TAXI_IN"])
df["ELAPSED_TIME_log"]    = np.log1p(df["ELAPSED_TIME"])
df["AIR_TIME_log"]        = np.log1p(df["AIR_TIME"])

In [ ]:
#check calculations
df.head()

### 1.6 Feature Removals

Here we remove redundant columns to further reduce the data size. Columns that are being removed:

- `YEAR`: All rows are from 2015, no need to include this.
- `AIRLINE`: We have AIRLINE_CODE which is the same information

In [ ]:
col_to_drop = ['YEAR','AIRLINE']
df = df.drop(columns = col_to_drop)

In [ ]:
df.head()

### 1.7 Encoding

We know that `TAIL_NUMBER`, `ORIGIN_AIRPORT`, and `DESTINATION_AIRPORT` contain a large number of unique values. Before proceeding, we wanted to check and see exactly how many of each we had.

In [ ]:
print(df['TAIL_NUMBER'].value_counts())
print(df['ORIGIN_AIRPORT'].value_counts())
print(df['DESTINATION_AIRPORT'].value_counts())

We have 3,700 different airplanes (TAIL_NUMBER) and 153 different airports (ORIGIN_AIRPORT and DESTINATION_AIRPORT). If we one-hot encode all of these, it would create way too many columns. We would also run the risk of not including a specific airport or tail number in our training data set which would cause an error if it is in our test data set. For that reason, let's set the cut-off to a minimum occurrence of 5 for tail numbers so that we have a greater likelihood of including it within either the train or test set. The minimum group size of the origin and destination airports is sufficiently large to avoid this problem.

In [ ]:
df = df[df.groupby('TAIL_NUMBER').TAIL_NUMBER.transform(len) > 4]
print(df['TAIL_NUMBER'].value_counts())

This now reduces to 3,228 unique tail numbers. Hopefully a minimum count of 5 does not give us errors later on.

Next, we will encode these variables. We are not one-hot encoding as there are still too many values. We know this will add some ordinality to the variables but it is simply not feasible to have 4000+ columns without moving to a cloud-based solution.

In [ ]:
%%time

from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()

df['ORIGIN_AIRPORT_encode'] = labelencoder.fit_transform(df['ORIGIN_AIRPORT'].astype('str'))
df2 = df[['ORIGIN_AIRPORT','ORIGIN_AIRPORT_encode']]
df2 = df2.drop_duplicates(subset=['ORIGIN_AIRPORT'], keep='last')

df2.rename(columns={'ORIGIN_AIRPORT': 'DESTINATION_AIRPORT'}, inplace=True)
df2.rename(columns={'ORIGIN_AIRPORT_encode': 'DESTINATION_AIRPORT_encode'}, inplace=True)

df = pd.merge(df, df2, on='DESTINATION_AIRPORT', how = 'left')
df.dropna(subset = ["DESTINATION_AIRPORT_encode"], inplace=True)

df['TAIL_NUMBER_encode'] = labelencoder.fit_transform(df['TAIL_NUMBER'])

In [ ]:
# check which value is DFW and DAL for later reference
df[(df.ORIGIN_AIRPORT == 'DFW') | (df.ORIGIN_AIRPORT == 'DAL')].head(10)

`DFW` is `37` and `DAL` is `33`. We also have code above which makes sure the `ORIGIN_AIRPORT` and `DESTINATION_AIRPORT` encodings are the same value by airport.

In [ ]:
# Drop original columns
col_to_drop = ['TAIL_NUMBER', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT']
df = df.drop(columns = col_to_drop)

For the remaining categorical variables, we can one-hot encode as the number of unique values is significantly fewer.

In [ ]:
df.info()

In [ ]:
#total cancel check
df['CANCELLED'].value_counts()

In [ ]:
%%time

categorical_columns = ['AIRLINE_CODE', 'CANCELLATION_REASON', 'SCHED_DEPARTURE_TIME', 
                       'ACTUAL_DEPARTURE_TIME','SCHED_ARRIVAL_TIME', 'ACTUAL_ARRIVAL_TIME',
                       'DISTANCE_BUCKET']

for column in categorical_columns:
  tempdf = pd.get_dummies(df[categorical_columns], prefix = categorical_columns, drop_first = True)
  df_OHE = pd.merge(
      left = df,
      right = tempdf,
      left_index=True,
      right_index=True
  )
  df_OHE = df_OHE.drop(columns = categorical_columns)

df_OHE.head()

In [ ]:
df_OHE.info()

In [ ]:
#add response variable bucket for delay time
delay_labels = ['Early_<0', 'On_Time_0-10', 'Late_11-30', 'Very_Late_31-60', 'Extremely_Late_61+']
delay_bins   = [-np.inf, 0, 10, 30, 60, np.inf]
df_OHE['DELAY_BUCKET'] = pd.cut(df_OHE['ARRIVAL_DELAY'],
                               bins=delay_bins,
                               labels=delay_labels)

#check counts by bucket
df_OHE['DELAY_BUCKET'].value_counts()

In [ ]:
#encode data set response variable
df_OHE['DELAY_BUCKET'] = labelencoder.fit_transform(df_OHE['DELAY_BUCKET'].astype('str'))

In [ ]:
#check counts by bucket
df_OHE['DELAY_BUCKET'].value_counts()

### 1.8 Make New Data Sets

Here, we will start to remove variables based on our desired response variable and then check correlations for further removals. We will drop the non-transformed versions of the below groups since it is duplicated and will naturally be correlated to the log version.

In [ ]:
col_to_drop = ['DEPARTURE_DELAY', 'ARRIVAL_DELAY', 'DISTANCE', 'TAXI_IN', 'ELAPSED_TIME', 'AIR_TIME']
df_OHE = df_OHE.drop(columns = col_to_drop)

In [ ]:
#make a copy of our data for each new set
df_cancel = df_OHE
df_delay = df_OHE

#drop delay bucket from cancellation set
df_cancel = df_cancel.drop(columns = 'DELAY_BUCKET')

#### 1.8.1 Delay Data Set

We can remove several variables for our delay bucket group. We will filter out any cancelled flights, as these are not delayed. Then we can also remove `CANCELLED` and our `CANCELLATION_REASON` encoded columns. Because we don't know whether or not the flight will be delayed prior to the analysis, we will also remove `ACTUAL_ARRIVAL_TIME` and `ACTUAL_DEPARTURE_TIME` and all of of our other delay time related columns.

In [ ]:
#filter out cancelled flights
df_delay = df_delay[df_delay.CANCELLED == 0]

col_to_drop2 = ['CANCELLED', 'CANCELLATION_REASON_B', 'CANCELLATION_REASON_C', 'CANCELLATION_REASON_N', 
               'ACTUAL_DEPARTURE_TIME_morning', 'ACTUAL_DEPARTURE_TIME_afternoon', 'ACTUAL_DEPARTURE_TIME_evening',
               'ACTUAL_DEPARTURE_TIME_N', 'ACTUAL_ARRIVAL_TIME_morning', 'ACTUAL_ARRIVAL_TIME_afternoon', 
                'SCHEDULED_DEPARTURE', 'SCHEDULED_ARRIVAL', 'AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 
                'ACTUAL_ARRIVAL_TIME_evening','ACTUAL_ARRIVAL_TIME_N','AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY', 'DELAYED', 
                'DEPARTURE_DELAY_log','ARRIVAL_DELAY_log', 'ELAPSED_TIME_log', 'DEPARTED', 'ARRIVED',
               'TAXI_IN_log', 'AIR_TIME_log']

df_delay = df_delay.drop(columns = col_to_drop2)

In [ ]:
df_delay.shape

In [ ]:
df_delay.info()

#### 1.8.2 Cancelled data set

In [ ]:
df_cancel.info()

For the cancellation data set, we need to remove the non-transformed versions of the variables we know will be correlated to their log version. 

In [ ]:
col_to_drop3 = ['DEPARTURE_DELAY_log','ARRIVAL_DELAY_log', 'ELAPSED_TIME_log',
               'TAXI_IN_log', 'AIR_TIME_log', 'DISTANCE_log']

df_cancel = df_cancel.drop(columns = col_to_drop3)

In [ ]:
#total cancel check
df_cancel['CANCELLED'].value_counts()

### 1.9 Check Correlations

#### 1.9.1 Delay Data Set

In [ ]:
# Examine correlation visually using Seaborn. 
# (Code adapted from 02. Data Visualization.ipynb)

cmap = sns.diverging_palette(220, 10, as_cmap=True)

sns.set(style = "darkgrid") # one of the many styles to plot using

f, ax = plt.subplots(figsize = (20, 20))

# Create heatmap
sns.heatmap(df_delay.corr(), cmap=cmap, annot = True)

For our delay data set, we still see a high correlation between `DISTANCE_log` and `SCHEDULED_TIME` at a value of 0.92. Let's remove the `DISTANCE_log` value as we have distance buckets already. Let's also remove `DIVERTED` as the heat map shows us we have all `0` values in this column so it's not useful.

In [ ]:
col_to_drop4 = ['DISTANCE_log', 'SCHEDULED_TIME', 'DIVERTED']

df_delay = df_delay.drop(columns = col_to_drop4)

#### 1.9.2 Cancel Data Set

In [ ]:
# Examine correlation visually using Seaborn. 
# (Code adapted from 02. Data Visualization.ipynb)

cmap = sns.diverging_palette(220, 10, as_cmap=True)

sns.set(style = "darkgrid") # one of the many styles to plot using

f, ax = plt.subplots(figsize = (20, 20))

# Create heatmap
sns.heatmap(df_cancel.corr(), cmap=cmap, annot = True)

In [ ]:
# Create correlation matrix with absolute values only
corr_matrix_abs = df_cancel.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix_abs.where(np.triu(np.ones(corr_matrix_abs.shape), k=1).astype(np.bool))

# Find features with correlation greater than 0.9
to_drop = [column for column in upper.columns if any(upper[column] > 0.9)]
print(to_drop)

For the columns that have a very high correlation, we will retain these columns for now due to their perceived importance in our analysis and will use feature selection techniques to remove if deemed necessary.

### 1.10 Final Data Sets

- **[5 points]** Describe the final dataset that is used for classification/regression (include a description of any newly formed variables you created).

In [ ]:
# #save data
# df_cancel.to_csv('../Data/df_cancel.csv', index=False)
# df_delay.to_csv('../Data/df_delay.csv', index=False)

In [ ]:
# #load data from here to save time
# df_cancel = pd.read_csv('../Data/df_cancel.csv')
# df_delay = pd.read_csv('../Data/df_delay.csv')

  
#### Modeling and Evaluation (70 points total)

- **[10 points]** Choose and explain your evaluation metrics that you will use (i.e., accuracy, precision, recall, F-measure, or any metric we have discussed). Why are the measure(s) appropriate for analyzing the results of your modeling? Give a detailed explanation backing up any assertions.
- **[10 points]** Choose the method you will use for dividing your data into training and testing splits (i.e., are you using Stratified 10-fold cross validation? Why?). Explain why your chosen method is appropriate or use more than one method as appropriate.
- **[20 points]** Create three different classification/regression models (e.g., random forest, KNN, and SVM). Two modeling techniques must be new (but the third could be SVM or logistic regression). Adjust parameters as appropriate to increase generalization performance using your chosen metric.
- **[10 points]** Analyze the results using your chosen method of evaluation. Use visualizations of the results to bolster the analysis. Explain any visuals and analyze why they are interesting to someone that might use this model.
- **[10 points]** Discuss the advantages of each model for each classification task, if any. If there are not advantages, explain why. Is any model better than another? Is the difference significant with 95% confidence? Use proper statistical comparison methods.
- **[10 points]** Which attributes from your analysis are most important? Use proper methods discussed in class to evaluate the importance of different attributes. Discuss the results and hypothesize about why certain attributes are more important than others for a given classification task.

#### Deployment (5 points total)

- **[5 points]** How useful is your model for interested parties (i.e., the companies or
organizations that might want to use it for prediction)? How would you measure the model's value if it was used by these parties? How would your deploy your model for interested parties? What other data should be collected? How often would the model need to be updated, etc.?

#### Exceptional Work (10 points total)

- You have free reign to provide additional modeling.
- One idea: grid search parameters in a parallelized fashion and visualize the performances across attributes. Which parameters are most significant for making a good model for each classification algorithm?